In [ ]:
import sys
sys.path.insert(0,"../")


sys.path.insert(0,"../src/")
import data_taking.all_channel as all_channel
import datetime
from IPython.display import clear_output
import os

In [ ]:
# User-defined parameters
comment = "put source" # optional; will be written into the metadata file

n_events = 10000 # must be >= 2000 due to noisy data in the beginning
# threshold_multiplier_list = [2.5, 3.0, 3.5, 4.0, 4.5, 5.0] # 46
# threshold_multiplier_list = [3.5, 4.0, 4.5, 5.0, 5.5, 6.0] # 47
# threshold_multiplier_list = [3.5, 4.0, 4.5, 5.0, 5.5] # 48
# threshold_multiplier_list = [4.0, 4.5, 5.0, 5.5, 6.0, 6.5] # 49
threshold_multiplier_list = [3.0] # 49
# threshold_multiplier_list = [4.0, 4.5, 5.0, 5.5, 6.0, 6.5] # 49
# threshold_multiplier_list = [4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0] # 50
# threshold_multiplier_list = [4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0] # 51
# threshold_multiplier_list = [2.5, 3.0] # test

bias_voltage = 46 #V
temperature = 98 #neg degC

# output_folder_base = f"/home/daqtest/DAQ/SandyAQ/softlink_to_data/all_data/20240828_test" # optional; if not specified, a folder will be created according to the bias_voltage and temperature
output_folder_base = f"/home/daqtest/DAQ/SandyAQ/softlink_to_data/all_data/20241018_all_3_T{temperature}_all_voltages" # optional; if not specified, a folder will be created according to the bias_voltage and temperature
# output_folder_base = f"/home/daqtest/DAQ/SandyAQ/softlink_to_data/all_data/20241017_test" # optional; if not specified, a folder will be created according to the bias_voltage and temperature

# if calibration = True:                redo calibration
# if calibration = a path to direction: will use the bin files from that folder as calibration
# if calibration = False:               manual input (see below)
# calibration = True
calibration = "/home/daqtest/DAQ/SandyAQ_vera/SandyAQ/softlink_to_data/all_data/20241018_all_T98_all_voltages_3.0sig/threshold_calibration" 

if not calibration: #specify the threshold values for each channel
    manual_threshold = {0:2359,1:2319,2:2618,3:2273,4:2426,5:2341,6:2468,7:2567,8:2801,9:2371,10:2210,
                11:2352,12:2470,13:2496,14:2602,15:2264,16:2284,17:2531,18:2216,19:2147,20:2522,
                21:2306,22:2369,23:2465} # optional; specify only when calibration = False
    input("Please confirm the manual_threshold values before proceeding. Press Enter to continue.")


# edit in case some channel is broken or no data, remove the channel from the list
channel_list = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]
# channel_list = [11]

# run tag can be a list of tags; will be joined by "|" into a string when running run_info_single_channel.py
# file path/name containing "trash" and "test" will automatically be tagged respectively
run_tag = "LXe/Cs137" # options: "GXe/gain_calibration", "LXe/gain_calibration", "vacuum/dark_count", "LXe/{}source"



################################################
# Create a test configuration; initialization

for run_count, threshold_multiplier in enumerate(threshold_multiplier_list):

    if output_folder_base[-1] == "/":
        output_folder_base = output_folder_base[:-1]
    
    output_folder = f"{output_folder_base}_{threshold_multiplier}sig"

    if n_events <2000:
        print("The number of events must be >= 2000")
        raise ValueError

    test_config = {}
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    test_config["comment"] = comment
    test_config["number_of_events"] = n_events
    test_config["run_tag"] = run_tag
    test_config["channel_list"] = channel_list

    if len(output_folder) > 0:
        test_config["output_folder"] = output_folder
    else:
        test_config["output_folder"] = f"./run{timestamp}_T{temperature}_V{bias_voltage}"
    test_config["voltage_config"] = {"preamp_1": -bias_voltage, "preamp_2": -bias_voltage,
                                    "preamp_3": -bias_voltage, "preamp_4": -bias_voltage,
                                    "preamp_5": -bias_voltage}
    test_config["temperature"] = -temperature

    if not calibration:
        test_config["channel_threshold_dict"] = manual_threshold
    
    if isinstance(calibration, str) or calibration == False:
        calibration = calibration
    elif calibration == True:
        # use the calibration from the first run
        if run_count == 0:
            calibration = calibration
            first_run_output_folder = output_folder
        else:
            # use the calibration from the first run
            calibration = os.path.join(first_run_output_folder, "threshold_calibration/")

    test_config["threshold_multiplier"] = threshold_multiplier
    test_calibration = all_channel.AllChannel_DataTaking(test_config, calibration=calibration, same_threshold=True)
    clear_output()
    test_calibration.run(dry_run=False)